In [181]:
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
from torch.optim import SGD


def usp(children_posteriors, parents_priors, thresh_a):
    parents_posteriors = []
    for i, p in enumerate(parents_priors):
        score = p
        for pc in children_posteriors[i]:
            if pc > thresh_a[i] and pc > score:
                score = pc
            else:
                delta = torch.max(torch.tensor(0), (pc - score) / thresh_a[i])
                score = score * torch.exp(delta)
                score = torch.min(torch.tensor(1), score)
        parents_posteriors.append(score)
    return torch.stack(parents_posteriors)




thresh_a = nn.Parameter(Tensor([0.5, 0.4]))
parents_priors = Tensor([0.15, 0.2])
children_posteriors = Tensor([[0.3, 0.2, 0.03], [0.1, 0.25, 0.1]])

targ = Tensor([1, 0])

criterion = nn.CrossEntropyLoss()
optimizer = SGD([thresh_a], lr=0.1)

In [182]:
for epoch in range(10):
    optimizer.zero_grad()
    y = usp(children_posteriors, parents_priors, thresh_a)
    loss = criterion(y.unsqueeze(0), targ.unsqueeze(0))
    loss.backward()
    optimizer.step()
    print('\nEpoch: %d, Loss: %.4f' % (epoch, loss.item()))
    print(thresh_a.data)
    print(y)


Epoch: 0, Loss: 0.7053
tensor([0.4939, 0.4036])
tensor([0.2025, 0.2266], grad_fn=<StackBackward0>)

Epoch: 1, Loss: 0.7048
tensor([0.4875, 0.4071])
tensor([0.2032, 0.2264], grad_fn=<StackBackward0>)

Epoch: 2, Loss: 0.7043
tensor([0.4810, 0.4105])
tensor([0.2040, 0.2261], grad_fn=<StackBackward0>)

Epoch: 3, Loss: 0.7037
tensor([0.4743, 0.4139])
tensor([0.2049, 0.2259], grad_fn=<StackBackward0>)

Epoch: 4, Loss: 0.7031
tensor([0.4674, 0.4173])
tensor([0.2058, 0.2257], grad_fn=<StackBackward0>)

Epoch: 5, Loss: 0.7025
tensor([0.4602, 0.4205])
tensor([0.2068, 0.2255], grad_fn=<StackBackward0>)

Epoch: 6, Loss: 0.7019
tensor([0.4528, 0.4237])
tensor([0.2078, 0.2253], grad_fn=<StackBackward0>)

Epoch: 7, Loss: 0.7012
tensor([0.4451, 0.4269])
tensor([0.2089, 0.2250], grad_fn=<StackBackward0>)

Epoch: 8, Loss: 0.7005
tensor([0.4371, 0.4300])
tensor([0.2101, 0.2249], grad_fn=<StackBackward0>)

Epoch: 9, Loss: 0.6998
tensor([0.4287, 0.4331])
tensor([0.2114, 0.2247], grad_fn=<StackBackward0>)
